### Setup Spark

In [1]:
import os
import pyspark
from pyspark.sql import SQLContext, SparkSession

# change "pierre.cs.colsotate.edu" to the name of your spark master node
SPARK_NODE="pierre.cs.colostate.edu"


# 31820 corresponds to SPARK_MASTER_PORT in $SPARK_HOME/conf/spark-env.sh
SPARK_PORT=31820

sc = SparkSession.builder.master('spark://{}:{}'.format(SPARK_NODE,SPARK_PORT)).appName('test').getOrCreate()

sqlContext = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)
print("Spark Version: " + sc.version)
print("PySpark Version: " + pyspark.__version__)

Spark Version: 3.0.3
PySpark Version: 3.0.3


In [2]:
# example load
metadata=sc.read.json("/FP/metadata.json")
metadata.show()

+---------+---------+--------------------+-------+-------+--------------------+--------------------+
|avgRating|dateAdded|          directedBy| imdbId|item_id|            starring|               title|
+---------+---------+--------------------+-------+-------+--------------------+--------------------+
|  3.89146|     null|       John Lasseter|0114709|      1|Tim Allen, Tom Ha...|    Toy Story (1995)|
|  3.26605|     null|        Joe Johnston|0113497|      2|Jonathan Hyde, Br...|      Jumanji (1995)|
|  3.17146|     null|       Howard Deutch|0113228|      3|Jack Lemmon, Walt...|Grumpier Old Men ...|
|  2.86824|     null|     Forest Whitaker|0114885|      4|Angela Bassett, L...|Waiting to Exhale...|
|   3.0762|     null|       Charles Shyer|0113041|      5|Steve Martin, Mar...|Father of the Bri...|
|  3.85549|     null|        Michael Mann|0113277|      6|Robert De Niro, A...|         Heat (1995)|
|  3.37244|     null|      Sydney Pollack|0114319|      7|Harrison Ford, Gr...|      Sabrin

In [5]:
df = sc.read.csv('/tmp/mnist_train.csv')
df.show()

+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

## Average ratings per user

In [3]:
# load data

ratings=sc.read.json("/FP/ratings.json")
ratings.show()


+-------+------+-------+
|item_id|rating|user_id|
+-------+------+-------+
|      5|   3.0| 997206|
|     10|   4.0| 997206|
|     13|   4.0| 997206|
|     17|   5.0| 997206|
|     21|   4.0| 997206|
|     28|   5.0| 997206|
|     31|   4.0| 997206|
|     39|   3.0| 997206|
|     40|   4.0| 997206|
|     45|   2.0| 997206|
|     46|   4.0| 997206|
|     50|   5.0| 997206|
|     62|   4.0| 997206|
|     74|   5.0| 997206|
|     85|   2.0| 997206|
|    110|   5.0| 997206|
|    112|   4.0| 997206|
|    141|   4.0| 997206|
|    150|   3.0| 997206|
|    161|   3.0| 997206|
+-------+------+-------+
only showing top 20 rows



In [19]:
# group by user id

ratings_mod_user_id = ratings.groupby("user_id").count()
total_ratings=ratings.count()
total_users=ratings_mod_user_id.count()

avg_ratings_per_user = total_ratings / total_users

In [20]:
print("total_ratings",total_ratings)
print("total_users",total_users)
print("avg_ratings_per_user",avg_ratings_per_user)

total_ratings 28490116
total_users 247383
avg_ratings_per_user 115.16602191743168


### Remove users with fewer than 5 ratings

In [24]:
ratings_mod_user_id = ratings.groupby("user_id").count()
more_than_four = ratings_mod_user_id.filter("count>4")


In [38]:
#more_than_four.select(show()

In [39]:
filtered_ratings = more_than_four.join(ratings,more_than_four.user_id == ratings.user_id).select(ratings.user_id,"item_id","rating")
#filtered_ratings.show()

In [36]:
print("before",total_ratings)
print("after",filtered_ratings.count())

before 28490116
after 28462184
